<a href="https://colab.research.google.com/github/Fulankeee/Credit-Risk-Early-Warning-Project/blob/main/notebooks/01_eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Preparation and Loading

In [5]:
# Install project dependencies
!pip install -r https://raw.githubusercontent.com/Fulankeee/Credit-Risk-Early-Warning-Project/main/requirements.txt

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# For data stored on google drive
from google.colab import drive
drive.mount('/content/drive')
DATA_PATH = "/content/drive/MyDrive/raw_data/"
# -----------------------------------------------------------------------------------------------------------------------------------
# For data stored on Onedrive

# -----------------------------------------------------------------------------------------------------------------------------------
# # Local saved dataset and local environment
# import os
# import pandas as pd
# DATA_PATH = os.getenv("DATA_PATH", "/Users/siyicheng/Desktop/raw_data")
# df = pd.read_csv(os.path.join(DATA_PATH, "application_train.csv"))
# -----------------------------------------------------------------------------------------------------------------------------------
# Upload directly to google colab
# from google.colab import files
# files.upload()

Mounted at /content/drive


In [7]:
app_train = pd.read_csv(DATA_PATH + "application_train.csv")
app_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
# Create DB + load CSVs
import os, sqlite3
import pandas as pd

DATA_DIR = "/content/drive/MyDrive/raw_data"
DB_PATH = "/content/home_credit.db"

conn = sqlite3.connect(DB_PATH)

files = [
    "application_train.csv",
    "application_test.csv",
    "bureau.csv",
    "bureau_balance.csv",
    "credit_card_balance.csv",
    "installments_payments.csv",
    "POS_CASH_balance.csv",
    "previous_application.csv"
]

for f in files:
    path = os.path.join(DATA_DIR, f)
    table = f.replace(".csv","")
    print("Loading:", table)
    df = pd.read_csv(path)
    df.to_sql(table, conn, if_exists="replace", index=False)

print("Done. Tables created.")

Loading: application_train
Loading: application_test
Loading: bureau
Loading: bureau_balance
Loading: credit_card_balance
Loading: installments_payments
Loading: POS_CASH_balance
Loading: previous_application
Done. Tables created.


In [9]:
# Sanity check
pd.read_sql("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;", conn)

,name
0,POS_CASH_balance
1,application_test
2,application_train
3,bureau
4,bureau_balance
5,credit_card_balance
6,installments_payments
7,previous_application


# Understand the tables

In [10]:
# Find the primary key 'SK_ID_CURR'

pd.read_sql("""
SELECT
  COUNT(*) AS n_rows,
  COUNT(DISTINCT SK_ID_CURR) AS n_customers
FROM application_train;
""", conn)

,n_rows,n_customers
0,307511,307511


In [11]:

pd.read_sql("""
SELECT
  AVG(CASE WHEN TARGET=1 THEN 1.0 ELSE 0 END) AS target_rate,
  SUM(CASE WHEN TARGET=1 THEN 1 ELSE 0 END) AS n_bad,
  COUNT(*) AS n_total
FROM application_train;
""", conn)


,target_rate,n_bad,n_total
0,0.080729,24825,307511


In [12]:
pd.read_sql("""
SELECT *
FROM application_train
LIMIT 5;
""", conn)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


# Create a clean, rational, time-aware SQL data model on top of the raw tables.

**Core cleaning rules**

- Convert common “missing” tokens to NULL: '', 'NA', 'NaN', 'null'

- Trim strings: TRIM(col)

- Cast numeric: CAST(col AS REAL/INTEGER)

- Parse dates if they exist (Home Credit often uses day offsets like DAYS_BIRTH, not real dates)

Table List:

- application_train

- application_test

- bureau

- bureau_balance

- credit_card_balance

- installments_payments

- POS_CASH_balance

- previous_application

- (sample_submission doesn’t matter for modeling)

- (HomeCredit_columns_description is metadata)

## application_train

In [13]:
pd.read_sql("PRAGMA table_info(application_train);", conn)

,cid,name,type,notnull,dflt_value,pk
0,0,SK_ID_CURR,INTEGER,0,None,0
1,1,TARGET,INTEGER,0,None,0
2,2,NAME_CONTRACT_TYPE,TEXT,0,None,0
3,3,CODE_GENDER,TEXT,0,None,0
4,4,FLAG_OWN_CAR,TEXT,0,None,0
...,...,...,...,...,...,...
117,117,AMT_REQ_CREDIT_BUREAU_DAY,REAL,0,None,0
118,118,AMT_REQ_CREDIT_BUREAU_WEEK,REAL,0,None,0
119,119,AMT_REQ_CREDIT_BUREAU_MON,REAL,0,None,0
120,120,AMT_REQ_CREDIT_BUREAU_QRT,REAL,0,None,0


In [14]:
pd.read_sql("""
SELECT *
FROM application_train
LIMIT 5;
""", conn)


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
# Clean table
cur = conn.cursor()
cur.execute("DROP VIEW IF EXISTS stg_application_train;")
conn.commit()

cur.execute("""
CREATE VIEW stg_application_train AS
SELECT
    -- IDs
    CAST(NULLIF(SK_ID_CURR, '') AS INTEGER) AS sk_id_curr,

    -- Target (train only)
    CAST(NULLIF(TARGET, '') AS INTEGER) AS target,

    -- Money fields
    CAST(NULLIF(AMT_INCOME_TOTAL, '') AS REAL) AS amt_income_total,
    CAST(NULLIF(AMT_CREDIT, '') AS REAL)       AS amt_credit,
    CAST(NULLIF(AMT_ANNUITY, '') AS REAL)      AS amt_annuity,

    -- Categorical cleanup
    NULLIF(TRIM(NAME_CONTRACT_TYPE), '') AS name_contract_type,
    NULLIF(TRIM(CODE_GENDER), '')        AS code_gender,

    -- Keep everything else for now
    *
FROM application_train;
""")

conn.commit()

print("stg_application_train created")

stg_application_train created


In [16]:
pd.read_sql("""
SELECT *
FROM stg_application_train
LIMIT 5;
""", conn)


,sk_id_curr,target,amt_income_total,amt_credit,amt_annuity,name_contract_type,code_gender,SK_ID_CURR:1,TARGET:1,NAME_CONTRACT_TYPE:1,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,202500.0,406597.5,24700.5,Cash loans,M,100002,1,Cash loans,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,270000.0,1293502.5,35698.5,Cash loans,F,100003,0,Cash loans,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,67500.0,135000.0,6750.0,Revolving loans,M,100004,0,Revolving loans,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,135000.0,312682.5,29686.5,Cash loans,F,100006,0,Cash loans,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,121500.0,513000.0,21865.5,Cash loans,M,100007,0,Cash loans,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


## bureau

In [17]:
pd.read_sql("""
SELECT *
FROM bureau
LIMIT 5;
""", conn)

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,None,0.0,Consumer credit,-131,None
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,None,0.0,Credit card,-20,None
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,None,0.0,Consumer credit,-16,None
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,None,0.0,Credit card,-16,None
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,None,0.0,Consumer credit,-21,None


In [30]:
# Clean table
cur = conn.cursor()
cur.execute("DROP VIEW IF EXISTS stg_bureau;")
conn.commit()

cur.execute("""
CREATE VIEW IF NOT EXISTS stg_bureau AS
SELECT
    CAST(SK_ID_CURR AS INTEGER) AS sk_id_curr,
    CAST(SK_ID_BUREAU AS INTEGER) AS sk_id_bureau,

    -- credit status
    NULLIF(TRIM(CREDIT_ACTIVE), '') AS credit_active,
    CAST(CREDIT_DAY_OVERDUE	AS INTEGER) AS credit_day_overdue,


    -- amounts
    CAST(NULLIF(AMT_CREDIT_SUM, '') AS REAL) AS amt_credit_sum,
    CAST(NULLIF(AMT_CREDIT_SUM_DEBT, '') AS REAL) AS amt_credit_sum_debt,
    CAST(NULLIF(AMT_CREDIT_SUM_OVERDUE, '') AS REAL) AS amt_credit_sum_overdue,

    -- timing
    CAST(NULLIF(DAYS_CREDIT, '') AS INTEGER) AS days_credit,
    CAST(NULLIF(DAYS_CREDIT_ENDDATE, '') AS INTEGER) AS days_credit_enddate

FROM bureau;

""")

conn.commit()

print("stg_bureau created")

stg_bureau created


In [31]:
pd.read_sql("""
SELECT *
FROM stg_bureau
LIMIT 5;
""", conn)


,sk_id_curr,sk_id_bureau,credit_active,credit_day_overdue,amt_credit_sum,amt_credit_sum_debt,amt_credit_sum_overdue,days_credit,days_credit_enddate
0,215354,5714462,Closed,0,91323.0,0.0,0.0,-497,-153.0
1,215354,5714463,Active,0,225000.0,171342.0,0.0,-208,1075.0
2,215354,5714464,Active,0,464323.5,NaN,0.0,-203,528.0
3,215354,5714465,Active,0,90000.0,NaN,0.0,-203,NaN
4,215354,5714466,Active,0,2700000.0,NaN,0.0,-629,1197.0


In [37]:
# AGG table
cur = conn.cursor()
cur.execute("""
CREATE TABLE agg_bureau AS
SELECT
    SK_ID_CURR,

    COUNT(*) AS bureau_loan_count,

    SUM(amt_credit_sum_overdue) AS bureau_total_days_overdue,
    AVG(amt_credit_sum_overdue) AS bureau_avg_days_overdue,

    SUM(amt_credit_sum) AS bureau_total_credit,
    SUM(amt_credit_sum_debt) AS bureau_total_debt,

    AVG(amt_credit_sum_debt / NULLIF(amt_credit_sum,0))
        AS bureau_avg_utilization,

    SUM(CASE WHEN credit_active = 'Active' THEN 1 ELSE 0 END)
        AS bureau_active_loans,

    SUM(CASE WHEN credit_active = 'Closed' THEN 1 ELSE 0 END)
        AS bureau_closed_loans

FROM stg_bureau
GROUP BY SK_ID_CURR;


""")

conn.commit()

print("agg_bureau created")

agg_bureau created


In [38]:
pd.read_sql("""
SELECT *
FROM agg_bureau
LIMIT 5;
""", conn)

,sk_id_curr,bureau_loan_count,bureau_total_days_overdue,bureau_avg_days_overdue,bureau_total_credit,bureau_total_debt,bureau_avg_utilization,bureau_active_loans,bureau_closed_loans
0,100001,7,0.0,0.0,1453365.000,596686.5,0.282518,3,4
1,100002,8,0.0,0.0,865055.565,245781.0,0.136545,2,6
2,100003,4,0.0,0.0,1017400.500,0.0,0.000000,1,3
3,100004,2,0.0,0.0,189037.800,0.0,0.000000,0,2
4,100005,3,0.0,0.0,657126.000,568408.5,0.601256,2,1


## previous_application

In [40]:
pd.read_sql("""
SELECT *
FROM previous_application
LIMIT 5;
""", conn)


,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,...,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,...,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,...,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,...,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,...,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
# Clean table
cur = conn.cursor()
cur.execute("DROP TABLE IF EXISTS stg_previous_application;")
conn.commit()

cur.execute("""
CREATE VIEW IF NOT EXISTS stg_previous_application AS
SELECT
    CAST(SK_ID_CURR AS INTEGER) AS sk_id_curr,
    CAST(SK_ID_PREV AS INTEGER) AS sk_id_prev,

    NULLIF(TRIM(NAME_CONTRACT_STATUS), '') AS contract_status,

    CAST(NULLIF(AMT_APPLICATION, '') AS REAL) AS amt_application,
    CAST(NULLIF(AMT_CREDIT, '') AS REAL) AS amt_credit,

    CAST(NULLIF(DAYS_DECISION, '') AS INTEGER) AS days_decision

FROM previous_application;


""")

conn.commit()

print("stg_previous_application created")

stg_previous_application created


In [42]:
pd.read_sql("""
SELECT *
FROM stg_previous_application
LIMIT 5;
""", conn)

,sk_id_curr,sk_id_prev,contract_status,amt_application,amt_credit,days_decision
0,271877,2030495,Approved,17145.0,17145.0,-73
1,108129,2802425,Approved,607500.0,679671.0,-164
2,122040,2523466,Approved,112500.0,136444.5,-301
3,176158,2819243,Approved,450000.0,470790.0,-512
4,202054,1784265,Refused,337500.0,404055.0,-781


In [44]:
# Clean table
cur = conn.cursor()
cur.execute("DROP TABLE IF EXISTS agg_previous_application;")
conn.commit()

cur.execute("""
CREATE TABLE agg_previous_application AS
SELECT
    SK_ID_CURR,

    COUNT(*) AS prev_app_count,

    AVG(amt_application) AS prev_avg_amt_applied,
    AVG(amt_credit) AS prev_avg_amt_approved,

    AVG(amt_credit / NULLIF(amt_application,0))
        AS prev_approval_ratio,

    MAX(DAYS_DECISION) AS prev_last_decision_days

FROM stg_previous_application
GROUP BY SK_ID_CURR;
""")

conn.commit()

print("agg_previous_application created")

agg_previous_application created


In [46]:
pd.read_sql("""
SELECT *
FROM agg_previous_application
LIMIT 5;
""", conn)


,sk_id_curr,prev_app_count,prev_avg_amt_applied,prev_avg_amt_approved,prev_approval_ratio,prev_last_decision_days
0,100001,1,24835.50,23787.00,0.957782,-1740
1,100002,1,179055.00,179055.00,1.000000,-606
2,100003,3,435436.50,484191.00,1.057664,-746
3,100004,1,24282.00,20106.00,0.828021,-815
4,100005,2,22308.75,20076.75,0.899950,-315


## Aggragate All Together

In [54]:
# Clean table
cur = conn.cursor()
cur.execute("DROP TABLE IF EXISTS feature_application_train;")
conn.commit()
cur.execute("""
CREATE TABLE feature_application_train AS
SELECT
    a.*,
    b.*,
    p.*

FROM stg_application_train a
LEFT JOIN agg_bureau b
    ON a.sk_id_curr = b.sk_id_curr
LEFT JOIN agg_previous_application p
    ON a.sk_id_curr = p.sk_id_curr;
""")

conn.commit()

print("feature_application_train created")

pd.read_sql("""
SELECT *
FROM feature_application_train
LIMIT 5;
""", conn)

feature_application_train created


,sk_id_curr,target,amt_income_total,amt_credit,amt_annuity,name_contract_type,code_gender,SK_ID_CURR:1,TARGET:1,NAME_CONTRACT_TYPE:1,...,bureau_total_debt,bureau_avg_utilization,bureau_active_loans,bureau_closed_loans,sk_id_curr:3,prev_app_count,prev_avg_amt_applied,prev_avg_amt_approved,prev_approval_ratio,prev_last_decision_days
0,100002,1,202500.0,406597.5,24700.5,Cash loans,M,100002,1,Cash loans,...,245781.0,0.136545,2.0,6.0,100002,1,179055.00,179055.00,1.000000,-606
1,100003,0,270000.0,1293502.5,35698.5,Cash loans,F,100003,0,Cash loans,...,0.0,0.000000,1.0,3.0,100003,3,435436.50,484191.00,1.057664,-746
2,100004,0,67500.0,135000.0,6750.0,Revolving loans,M,100004,0,Revolving loans,...,0.0,0.000000,0.0,2.0,100004,1,24282.00,20106.00,0.828021,-815
3,100006,0,135000.0,312682.5,29686.5,Cash loans,F,100006,0,Cash loans,...,NaN,NaN,NaN,NaN,100006,9,272203.26,291695.50,1.012684,-181
4,100007,0,121500.0,513000.0,21865.5,Cash loans,M,100007,0,Cash loans,...,0.0,0.000000,0.0,1.0,100007,6,150530.25,166638.75,1.046356,-374
